In [1]:
import os
import re
import sys
import glob
import math
import logging
from pathlib import Path
from pprint import pprint

import numpy as np
import scipy as sp
import sklearn

import spacy
import tika
from tika import parser

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from wb_nlp.processing import document

In [3]:
## Hints

# nlp = spacy.load('en_core_web_sm')

This notebook contains examples of how the `PDFDoc2Txt` class can be used to convert pdf documents into formatted text. Additional methods implemented in this class can also be applied to raw texts extracted from PDFs.

We start by creating an instance of the `PDFDoc2Txt`—`pdf_parser`.

In [4]:
pdf_parser = document.PDFDoc2Txt()

# Parsing a pdf file

Parsing a pdf file starts with the `parse` method. This method accepts a buffer of byte object or a string to a url or file path. The source type must be specified for the parser to correctly execute the processing.

In [5]:
??pdf_parser.parse

Signature: pdf_parser.parse(source: Union[bytes, str], source_type: str = 'buffer') -> str
Source:   
    def parse(self, source: Union[bytes, str], source_type: str='buffer') -> str:
        """Parse a PDF document to text from different source types.

        Args:
            source:
                Source of the PDF that needs to be converted.
                The source could be a url, a path, or a buffer/file-like object
                to the PDF file.
            source_type:
                Specification of which source type is being provided in `source`.

        Returns:
            A string containing the parsed pdf file.

        """
        if source_type == 'url':
            buf = requests.get(source)
            pdf_text = parser.from_buffer(buf.content, xmlContent=True)

        elif source_type == 'file':
            pdf_text = parser.from_file(source, xmlContent=True)

        elif source_type == 'buffer':
            pdf_text = parser.from_buffer(source, xmlContent=

## Processing a single page

In [6]:
??pdf_parser.process_page

Signature: pdf_parser.process_page(page: bs4.element.Tag) -> str
Docstring: <no docstring>
Source:   
    @staticmethod
    def process_page(page: bs4.element.Tag) -> str:
        paragraphs = []

        for p in page.find_all('p'):
            paragraph = PDFDoc2Txt.consolidate_paragraph(p.text)
            paragraph = PDFDoc2Txt.normalize_footnote_citations(paragraph)
            if not paragraph:
                continue

            prev_paragraph_end = paragraphs[-1][-1] if paragraphs and paragraphs[-1] else ''

            if prev_paragraph_end and (re.search(r'[a-zA-Z\-\,]', prev_paragraph_end) or (paragraph[0].islower())):  # .isalpha():
                paragraph = paragraphs[-1] + ' ' + paragraph
                paragraphs[-1] = paragraph
            else:
                paragraphs.append(paragraph)

        # for p in paragraphs:
        #     doc = self.nlp(p)
        #     self.sentences.extend(list(doc.sents))

        paragraphs = '\n\n'.join(paragraphs)
        return 

# Paragraph consolidation algorithm

In [7]:
??pdf_parser.consolidate_paragraph

Signature:
pdf_parser.consolidate_paragraph(
    text_paragraph: str,
    min_fragment_len: int = 3,
) -> str
Source:   
    @staticmethod
    def consolidate_paragraph(text_paragraph: str, min_fragment_len: int=3) -> str:
        """Consolidate a `text_paragraph` with possible multiple newlines into one logical paragraph.

        Tika provides access to extracted text by paragraph. These paragraphs, however, may contain
        multiple newlines that break the paragraph arbitrarily. This function implements some heuristics
        to recover a logical representation of the paragraph.

        Args:
            text_paragraph:
                Text corresponding to an extracted paragraph from Tika.

        Returns:
            A string corresponding to a logical paragraph.

        """
        replace_chars = {'’': "'", '“': '"', '”': '"'}
        line_seps = set([' ', '-'])
        line_groups = [[]]

        text_paragraph = text_paragraph.replace('\r', '')

        lines = []
     